In [1]:
import numpy as np
import pandas as pd
import spacy
from spellchecker import SpellChecker
from nltk.tokenize import wordpunct_tokenize
import nltk
from nltk.stem.lancaster import LancasterStemmer
from encoder import label_encoder
import pickle
import json
import tflearn
import tensorflow as tf
import random

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


C:\Anaconda\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [2]:
nlp = spacy.load("en_core_web_md")

In [3]:
def autocorrect(text):
    spell = SpellChecker()
    
    low = wordpunct_tokenize(text)
    corrected = ""
    punctuations = """!"#$%&'()*+, -./:;<=>?@[\]^_`{|}~"""
    
    for word in low:
        if word not in punctuations:
            if (corrected != "") and (corrected[-1] != "'"):
                corrected += " " + spell.correction(word)
            else:
                corrected += spell.correction(word)
        else:
            corrected += word
    return corrected

In [14]:
def hasSymptoms(text):
    low = text.replace(',',' ').split()
    df = pd.read_csv("Database/Symptom-severity.csv")
    df['Symptom'] = df['Symptom'].str.replace(' ','')
    df['Symptom'] = df['Symptom'].str.replace('_',' ')
    possibility = 0
    for word in low:
        similarities = {}
        for symptom in df['Symptom'].values:
            similarities[symptom] = nlp(word).similarity(nlp(symptom))
        possibility = max(max(similarities.values()),possibility)
    return possibility > 0.55

In [5]:
def strip(text):

    stop_words = ['a',
                     'about',
                     'above',
                     'after',
                     'again',
                     'against',
                     'ain',
                     'all',
                     'am',
                     'an',
                     'any',
                     'are',
                     'aren',
                     "aren't",
                     'as',
                     'be',
                     'because',
                     'been',
                     'before',
                     'being',
                     'below',
                     'between',
                     'both',
                     'but',
                     'by',
                     'can',
                     'couldn',
                     "couldn't",
                     'd',
                     'did',
                     'didn',
                     "didn't",
                     'do',
                     'does',
                     'doesn',
                     "doesn't",
                     'doing',
                     'don',
                     "don't",
                     'down',
                     'during',
                     'each',
                     'few',
                     'for',
                     'from',
                     'further',
                     'had',
                     'hadn',
                     "hadn't",
                     'has',
                     'hasn',
                     "hasn't",
                     'have',
                     'haven',
                     "haven't",
                     'having',
                     'he',
                     'her',
                     'here',
                     'hers',
                     'herself',
                     'him',
                     'himself',
                     'his',
                     'how',
                     'i',
                     'if',
                     'into',
                     'is',
                     'isn',
                     "isn't",
                     'it',
                     "it's",
                     'its',
                     'itself',
                     'just',
                     'll',
                     'm',
                     'ma',
                     'me',
                     'mightn',
                     "mightn't",
                     'more',
                     'most',
                     'mustn',
                     "mustn't",
                     'my',
                     'myself',
                     'needn',
                     "needn't",
                     'no',
                     'nor',
                     'not',
                     'now',
                     'o',
                     'of',
                     'off',
                     'once',
                     'only',
                     'or',
                     'other',
                     'our',
                     'ours',
                     'ourselves',
                     'out',
                     'over',
                     'own',
                     're',
                     's',
                     'same',
                     'shan',
                     "shan't",
                     'she',
                     "she's",
                     'should',
                     "should've",
                     'shouldn',
                     "shouldn't",
                     'so',
                     'some',
                     'such',
                     't',
                     'than',
                     'that',
                     "that'll",
                     'the',
                     'their',
                     'theirs',
                     'them',
                     'themselves',
                     'then',
                     'there',
                     'these',
                     'they',
                     'this',
                     'those',
                     'through',
                     'to',
                     'too',
                     'under',
                     'until',
                     'up',
                     've',
                     'very',
                     'was',
                     'wasn',
                     "wasn't",
                     'we',
                     'were',
                     'weren',
                     "weren't",
                     'what',
                     'when',
                     'where',
                     'which',
                     'while',
                     'who',
                     'whom',
                     'why',
                     'will',
                     'with',
                     'won',
                     "won't",
                     'wouldn',
                     "wouldn't",
                     'y',
                     'you',
                     "you'd",
                     "you'll",
                     "you're",
                     "you've",
                     'your',
                     'yours',
                     'yourself',
                     'yourselves']
    
    punctuations = """!"#$%&'()*+ -./:;<=>?@[\]^_`{|}~"""
    

    word_tokens = wordpunct_tokenize(text)

    filtered_sentence = [w for w in word_tokens if (not w.lower() in stop_words) and (w not in punctuations)]
    
    return " ".join(filtered_sentence)

In [6]:
def symptomize(text):
    los = text.replace(' and ',' , ').split(',')
    df = pd.read_csv("Database/Symptom-severity.csv")
    df['Symptom'] = df['Symptom'].str.replace(' ','')
    df['Symptom'] = df['Symptom'].str.replace('_',' ')
    
    formatted = []
    
    for each in los:
        if each[0] == " ":
            each = each[1:]
        if each[-1] == " ":
            each = each[:-1]
        similarities = {}
        for symptom in df['Symptom'].values:
            similarities[symptom] = nlp(each).similarity(nlp(symptom))
        keymax = "Null" if max(similarities.values()) == 0 else max(similarities, key= lambda x: similarities[x])
        if keymax != "Null":
            formatted.append(keymax.replace(' ','_'))
    
    return formatted

In [7]:
data = pickle.load(open("training_data","rb"))

words = data["words"]
classes = data["classes"]
train_x = data["train_x"]
train_y = data["train_y"]


with open("intents.json") as json_data:
    intents = json.load(json_data)

In [8]:
# Loading TFLearn Model


net = tflearn.input_data(shape = [None, len(train_x[0])])
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,len(train_y[0]),activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net,tensorboard_dir='tflearn_logs')
model.load('model.tflearn')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Restoring parameters from D:\My Documents\COLLEGE\Major Project\Code\ML Model\model.tflearn


In [9]:

stemmer = LancasterStemmer()



In [10]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    
    return sentence_words

def bow(sentence,words,show_details=False):
    sentence_words = clean_up_sentence(sentence)
    
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print("found in bag : %s" % w)
                    
    return(np.array(bag))

In [16]:
bow("Can you help me?",words)

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1])

In [11]:
ERROR_THRESHOLD = 0.80
def classify(sentence):
    results = model.predict([bow(sentence,words)])[0]
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1],reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]],r[1]))
        
    return return_list

def response(sentence,show_details=False):
    results = classify(sentence)
    
    if results:
        while results:
            for i in intents['intents']:
                if i['tag'] == results[0][0]:
                    return random.choice(i['responses'])
                
            results.pop(0)
            
    else:
        for i in intents['intents']:
            if i['tag'] == "error":
                return i['responses'][0]

In [62]:
def pipeline(text):
    
    # Convert to lower case
    text = text.lower()
    
    # Correct spelling errors
    text = autocorrect(text)
    
    output = response(text)
    
    if output.split()[0] == 'Sorry,':
    
        if hasSymptoms(text):

            # Preprocessing text
            text = strip(text)

            # Symptom mapper
            ls = symptomize(text)

            # Encoder
            array_of_symptoms = label_encoder(ls)

            return array_of_symptoms
    
    return(output)

In [70]:
type("esdjsk") == str

True

In [3]:
ls = [1,2,3]
g = str(ls)

In [6]:
g = g[1:-1]

In [7]:
print(g)

1, 2, 3


In [8]:
l = g.split(',')

In [9]:
l

['1', ' 2', ' 3']

In [10]:
l = [int(x) for x in l]
print(l)

[1, 2, 3]


In [15]:
x = "global"

def display():
    global x
    x = "mondial"
    if True:
        print("Hey this is just a random line to check mandate of continuity")
        
    x = "vaishvik"
    print(x)

display()
print(x)

Hey this is just a random line to check mandate of continuity
vaishvik
vaishvik


In [14]:
deja = list()
print(deja)

[]


In [16]:
og = [5,6,7,0,0,0,0]

In [17]:
invader = [8,9,10,0,0,0,0]

In [18]:
og.extend(invader)

In [19]:
og

[5, 6, 7, 0, 0, 0, 0, 8, 9, 10, 0, 0, 0, 0]

In [ ]:
sort(og)

## test


In [12]:
response("No")

"Sorry, I don't understand. Please re-enter your query"

In [15]:
hasSymptoms("no")

C:\Users\Shreyan Das\AppData\Local\Temp\ipykernel_8544\1342047661.py:10: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarities[symptom] = nlp(word).similarity(nlp(symptom))


0.543751591610493

In [18]:
disease="Fungal infection"
descr = pd.read_csv("Database/symptom_Description.csv")
prec = pd.read_csv("Database/symptom_precaution.csv")


In [20]:
prec['Precaution_1']

0                          stop irritation
1                 Consult nearest hospital
2                           apply calamine
3                            reduce stress
4         wash hands with warm soapy water
5                   avoid fatty spicy food
6                               cold baths
7                 Consult nearest hospital
8                            acetaminophen
9                                 lie down
10                        lie down on side
11                              bath twice
12                      have balanced diet
13        soak affected area in warm water
14                              meditation
15                  avoid fatty spicy food
16                  avoid fatty spicy food
17             drink vitamin c rich drinks
18                    use neem in bathing 
19            use heating pad or cold pack
20                             eat healthy
21                   drink plenty of water
22    lie down flat and raise the leg high
23         